# 라이브러리 호출

In [1]:
import numpy as np # Numpy
import pandas as pd # Pandas
import matplotlib as mpl #Matplotlib 세팅용
import matplotlib.pyplot as plt # 시각화 도구
import seaborn as sns # 시각화 도구
from sklearn.model_selection import train_test_split # 데이터셋 분리
from sklearn.model_selection import KFold # KFold 교차검증
from sklearn.cluster import KMeans # 클러스터링
from sklearn.metrics import silhouette_score # 실루엣 점수
import xgboost as xgb # XGBoost
from sklearn.model_selection import GridSearchCV # 그리드 서치
from sklearn.metrics import accuracy_score, precision_score # 평가 지표
from sklearn.metrics import recall_score, confusion_matrix, roc_auc_score, f1_score # 평가 지표
from imblearn.combine import SMOTEENN, SMOTETomek # 복합샘플링
from hyperopt import hp, fmin, tpe, Trials # HyperOPT
from nltk.corpus import names # nltk
import nltk
nltk.download("names")
from nltk import NaiveBayesClassifier
from scipy import stats
from collections import Counter
import random

import warnings # 경고문 제거용


%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

# 한글 폰트 설정
mpl.rc('font', family='D2Coding')
# 유니코드에서 음수 부호 설정
mpl.rc('axes', unicode_minus = False)

warnings.filterwarnings('ignore')
sns.set(font="D2Coding", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc('figure', figsize=(10,8))

[nltk_data] Downloading package names to
[nltk_data]     C:\Users\Windows\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!


# 데이터로딩

In [2]:
data = pd.read_excel('train_test_na_filled.xlsx', sheet_name='Train')
data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin1,Cabin2,Combi,Cabin3,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B,0.0,B0,P,B/0/P,TRAPPIST-1e,39,False,0,0,0,0,0,Maham Ofracculy,False
1,0002_01,Earth,False,F,0.0,F0,S,F/0/S,TRAPPIST-1e,24,False,109,9,25,549,44,Juanna Vines,True
2,0003_01,Europa,False,A,0.0,A0,S,A/0/S,TRAPPIST-1e,58,True,43,3576,0,6715,49,Altark Susent,False
3,0003_02,Europa,False,A,0.0,A0,S,A/0/S,TRAPPIST-1e,33,False,0,1283,371,3329,193,Solam Susent,False
4,0004_01,Earth,False,F,1.0,F1,S,F/1/S,TRAPPIST-1e,16,False,303,70,151,565,2,Willy Santantines,True


In [3]:
test = pd.read_excel('train_test_na_filled.xlsx', sheet_name='Test')
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin1,Cabin2,Combi,Cabin3,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0013_01,Earth,True,G,3.0,G3,S,G/3/S,TRAPPIST-1e,27,False,0,0,0,0,0,Nelly Carsoning,NaN
1,0018_01,Earth,False,F,4.0,F4,S,F/4/S,TRAPPIST-1e,19,False,0,9,0,2823,0,Lerome Peckers,NaN
2,0019_01,Europa,True,C,0.0,C0,S,C/0/S,55 Cancri e,31,False,0,0,0,0,0,Sabih Unhearfus,NaN
3,0021_01,Europa,False,C,1.0,C1,S,C/1/S,TRAPPIST-1e,38,False,0,6652,0,181,585,Meratz Caltilter,NaN
4,0023_01,Earth,False,F,5.0,F5,S,F/5/S,TRAPPIST-1e,20,False,10,0,635,0,0,Brence Harperez,NaN


In [4]:
print(data.shape)
print(test.shape)

(8693, 18)
(4277, 18)


In [5]:
data.drop('Cabin2', inplace=True, axis=1)

In [6]:
test.drop('Cabin2', inplace=True, axis=1)

# 이상치 확인 및 제거

In [7]:
def outlier_detection_train(df, n, columns):
    rows = []
    will_drop_train = []
    for col in columns:
        Q1 = np.nanpercentile(data[col], 25)
        Q3 = np.nanpercentile(data[col], 75)
        IQR = Q3 - Q1
        outlier_point = 1.5 * IQR
        rows.extend(df[(df[col] < Q1 - outlier_point)|(df[col] > Q3 + outlier_point)].index)
    for r, c in Counter(rows).items():
        if c >= n: will_drop_train.append(r)
    return will_drop_train

In [8]:
will_drop_train = outlier_detection_train(data, 5 ,data.select_dtypes(["float", "int"]).columns)
will_drop_train

[338,
 1390,
 6469,
 7038,
 1936,
 3317,
 3980,
 4762,
 6509,
 7007,
 7065,
 7294,
 7689,
 7957,
 8064]

In [9]:
data.drop(will_drop_train, inplace = True, axis = 0)

In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4277 non-null   object 
 2   CryoSleep     4277 non-null   bool   
 3   Cabin1        4214 non-null   object 
 4   Combi         4214 non-null   object 
 5   Cabin3        4214 non-null   object 
 6   Cabin         4214 non-null   object 
 7   Destination   4277 non-null   object 
 8   Age           4277 non-null   int64  
 9   VIP           4277 non-null   bool   
 10  RoomService   4277 non-null   int64  
 11  FoodCourt     4277 non-null   int64  
 12  ShoppingMall  4277 non-null   int64  
 13  Spa           4277 non-null   int64  
 14  VRDeck        4277 non-null   int64  
 15  Name          4183 non-null   object 
 16  Transported   0 non-null      float64
dtypes: bool(2), float64(1), int64(6), object(8)
memory usage: 509.7+ KB


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8678 entries, 0 to 8692
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PassengerId   8678 non-null   object
 1   HomePlanet    8678 non-null   object
 2   CryoSleep     8678 non-null   bool  
 3   Cabin1        8575 non-null   object
 4   Combi         8575 non-null   object
 5   Cabin3        8575 non-null   object
 6   Cabin         8575 non-null   object
 7   Destination   8678 non-null   object
 8   Age           8678 non-null   int64 
 9   VIP           8678 non-null   bool  
 10  RoomService   8678 non-null   int64 
 11  FoodCourt     8678 non-null   int64 
 12  ShoppingMall  8678 non-null   int64 
 13  Spa           8678 non-null   int64 
 14  VRDeck        8678 non-null   int64 
 15  Name          8478 non-null   object
 16  Transported   8678 non-null   bool  
dtypes: bool(3), int64(6), object(8)
memory usage: 1.0+ MB


In [12]:
test['Transported']= test['Transported'].astype('bool')

# 새로운 feature 생성

## 총 사용금액, 그리고 사용한 금액에 따라 poor, middle, rich로 분류

In [13]:
data["Total"] = data["RoomService"] + data["FoodCourt"] + data["ShoppingMall"] + data["Spa"] + data["VRDeck"] 
data["RichPoor"] = data["Total"].apply(lambda x: "poor" if x < 5000\
                                       else ("middle" if x>=5000 and x<20000 else "rich"))

test["Total"] = test["RoomService"] + test["FoodCourt"] + test["ShoppingMall"] + test["Spa"] + test["VRDeck"] 
test["RichPoor"] = test["Total"].apply(lambda x: "poor" if x < 5000\
                                       else ("middle" if x>=5000 and x<20000 else "rich"))

## 그룹여행객 여부

In [14]:
data["GroupId"] = data["PassengerId"].apply(lambda x: x.split("_")[0])
test["GroupId"] = test["PassengerId"].apply(lambda x: x.split("_")[0])
data["GroupNo"] = data["PassengerId"].apply(lambda x: x.split("_")[1])
test["GroupNo"] = test["PassengerId"].apply(lambda x: x.split("_")[1])

train_g = data[data["GroupId"].duplicated()]["GroupId"]
test_g = test[test["GroupId"].duplicated()]["GroupId"]
data["Group"] = data["GroupId"].apply(lambda x: x in train_g.values)
test["Group"] = test["GroupId"].apply(lambda x: x in test_g.values)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8678 entries, 0 to 8692
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PassengerId   8678 non-null   object
 1   HomePlanet    8678 non-null   object
 2   CryoSleep     8678 non-null   bool  
 3   Cabin1        8575 non-null   object
 4   Combi         8575 non-null   object
 5   Cabin3        8575 non-null   object
 6   Cabin         8575 non-null   object
 7   Destination   8678 non-null   object
 8   Age           8678 non-null   int64 
 9   VIP           8678 non-null   bool  
 10  RoomService   8678 non-null   int64 
 11  FoodCourt     8678 non-null   int64 
 12  ShoppingMall  8678 non-null   int64 
 13  Spa           8678 non-null   int64 
 14  VRDeck        8678 non-null   int64 
 15  Name          8478 non-null   object
 16  Transported   8678 non-null   bool  
 17  Total         8678 non-null   int64 
 18  RichPoor      8678 non-null   object
 19  GroupI

In [16]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PassengerId   4277 non-null   object
 1   HomePlanet    4277 non-null   object
 2   CryoSleep     4277 non-null   bool  
 3   Cabin1        4214 non-null   object
 4   Combi         4214 non-null   object
 5   Cabin3        4214 non-null   object
 6   Cabin         4214 non-null   object
 7   Destination   4277 non-null   object
 8   Age           4277 non-null   int64 
 9   VIP           4277 non-null   bool  
 10  RoomService   4277 non-null   int64 
 11  FoodCourt     4277 non-null   int64 
 12  ShoppingMall  4277 non-null   int64 
 13  Spa           4277 non-null   int64 
 14  VRDeck        4277 non-null   int64 
 15  Name          4183 non-null   object
 16  Transported   4277 non-null   bool  
 17  Total         4277 non-null   int64 
 18  RichPoor      4277 non-null   object
 19  GroupI

## 나이브 베어스를 활용한 이름을 통한 성별 분류

In [17]:
# Train_Data 이름과 성 분리
names_train_data = []
for n in data["Name"]:
    n = str(n)
    a = n.split()
    names_train_data.append(a[0])

In [18]:
# NLTK의 names 파일을 활용하여 이름을 여성과 남성으로 분리
labeled_names = [(name, "female") for name in names.words("female.txt")] + \
[(name, "male") for name in names.words("male.txt")]
random.shuffle(labeled_names)

In [19]:
# 이름의 마지막 단어 가져오는 함수
def gender_features(word):
    return {'last_letter': word[-1]}

In [20]:
names_train_data

['Maham',
 'Juanna',
 'Altark',
 'Solam',
 'Willy',
 'Sandie',
 'Billex',
 'Candra',
 'Andona',
 'Erraiam',
 'Altardr',
 'Wezena',
 'Berers',
 'Reney',
 'Elle',
 'Justie',
 'Flats',
 'Carry',
 'Alus',
 'Lyde',
 'Philda',
 'Almary',
 'Glendy',
 'Mollen',
 'Breney',
 'Mael',
 'Terta',
 'nan',
 'Penton',
 'Karard',
 'Anyoni',
 'Ceros',
 'Ginia',
 'Coobix',
 'Cinets',
 'Dontch',
 'Ziba',
 'Luse',
 'Marina',
 'Loise',
 'Jorgie',
 'Margia',
 'Ankalik',
 'Jodye',
 'Kayne',
 'Cassa',
 'Zelowl',
 'Mass',
 'Sony',
 'Vivia',
 'Elaney',
 'Elson',
 'Okulas',
 'Instab',
 'Zinoces',
 'Warry',
 'Shanya',
 'Sterry',
 'nan',
 'Colatz',
 'Diandy',
 'Ninaha',
 'Celine',
 'Velyne',
 'Cinst',
 'nan',
 'Meremy',
 'Nelly',
 'Thell',
 'Gorn',
 'Aldibah',
 'Conk',
 'Pon',
 'Spuri',
 'Dellie',
 'Totse',
 'Eaturs',
 'nan',
 'Coren',
 'Furudah',
 'Jodye',
 'Stmeal',
 'Heremy',
 'Deanne',
 'Tinez',
 'Gracy',
 'Stald',
 'Tiney',
 'Alchium',
 'Doria',
 'Leence',
 'Aliey',
 'Thewis',
 'Book',
 'Ritany',
 'Arlen',
 'Sh

In [21]:
# 나이브 베어스 모델 학습
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
classifier = NaiveBayesClassifier.train(featuresets)

In [22]:
# 성별 feature 생성
names_gender = []
for i in names_train_data:
    names_gender.append(classifier.classify(gender_features(i)))

# create new column called 'gender'    
data["Gender"] = names_gender

In [23]:
data.Gender[data.Name.isna()] = 'female'

In [24]:
data.Gender[data.Name.isna()].unique()

array(['female'], dtype=object)

In [25]:
# 테스트 세트 이름과 성 분리
names_test_data = []
for i in test["Name"]:
    i = str(i)
    a = i.split()
    names_test_data.append(a[0])

In [26]:
# NLTK의 names 파일을 활용하여 이름을 여성과 남성으로 분리
labeled_names = [(name, "female") for name in names.words("female.txt")] + \
[(name, "male") for name in names.words("male.txt")]
random.shuffle(labeled_names)

In [27]:
# 나이브 베어스 모델 학습
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
classifier = NaiveBayesClassifier.train(featuresets)

In [28]:
# 이름의 마지막 단어 가져오는 함수
def gender_features(word):
    return {'last_letter': word[-1]}

In [29]:
# 성별 feature 생성
names_gender = []
for i in names_test_data:
    names_gender.append(classifier.classify(gender_features(i)))

In [30]:
test["Gender"] = pd.Series(names_gender)

In [31]:
test.Gender[test.Name.isna()] = 'female'

In [32]:
data.Gender[data.Name.isna()].unique()

array(['female'], dtype=object)

## 연령 분류 

In [33]:
def get_score(v):
    if v <= 16:
        score = 0
    elif v <= 26:
        score = 1
    elif v <= 36:
        score = 2
    elif v <= 62:
        score = 3
    else:
        score = 4
    return score

# data 데이터 세트 진행
data["Age"] = data["Age"].apply(lambda v: get_score(v))
data

,PassengerId,HomePlanet,CryoSleep,Cabin1,Combi,Cabin3,Cabin,Destination,Age,VIP,...,Spa,VRDeck,Name,Transported,Total,RichPoor,GroupId,GroupNo,Group,Gender
0,0001_01,Europa,False,B,B0,P,B/0/P,TRAPPIST-1e,3,False,...,0,0,Maham Ofracculy,False,0,poor,0001,01,False,male
1,0002_01,Earth,False,F,F0,S,F/0/S,TRAPPIST-1e,1,False,...,549,44,Juanna Vines,True,736,poor,0002,01,False,female
2,0003_01,Europa,False,A,A0,S,A/0/S,TRAPPIST-1e,3,True,...,6715,49,Altark Susent,False,10383,middle,0003,01,True,male
3,0003_02,Europa,False,A,A0,S,A/0/S,TRAPPIST-1e,2,False,...,3329,193,Solam Susent,False,5176,middle,0003,02,True,male
4,0004_01,Earth,False,F,F1,S,F/1/S,TRAPPIST-1e,0,False,...,565,2,Willy Santantines,True,1091,poor,0004,01,False,female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A,A98,P,A/98/P,55 Cancri e,3,True,...,1643,74,Gravior Noxnuther,False,8536,middle,9276,01,False,male
8689,9278_01,Earth,True,G,G1499,S,G/1499/S,PSO J318.5-22,1,False,...,0,0,Kurta Mondalley,False,0,poor,9278,01,False,female
8690,9279_01,Earth,False,G,G1500,S,G/1500/S,TRAPPIST-1e,1,False,...,1,0,Fayey Connon,True,1873,poor,9279,01,False,female
8691,9280_01,Europa,False,E,E608,S,E/608/S,55 Cancri e,2,False,...,353,3235,Celeon Hontichre,False,4637,poor,9280,01,True,male


In [34]:
# 테스트 데이터세트 진행
test["Age"] = test["Age"].apply(lambda v: get_score(v))
test

,PassengerId,HomePlanet,CryoSleep,Cabin1,Combi,Cabin3,Cabin,Destination,Age,VIP,...,Spa,VRDeck,Name,Transported,Total,RichPoor,GroupId,GroupNo,Group,Gender
0,0013_01,Earth,True,G,G3,S,G/3/S,TRAPPIST-1e,2,False,...,0,0,Nelly Carsoning,True,0,poor,0013,01,False,female
1,0018_01,Earth,False,F,F4,S,F/4/S,TRAPPIST-1e,1,False,...,2823,0,Lerome Peckers,True,2832,poor,0018,01,False,female
2,0019_01,Europa,True,C,C0,S,C/0/S,55 Cancri e,2,False,...,0,0,Sabih Unhearfus,True,0,poor,0019,01,False,female
3,0021_01,Europa,False,C,C1,S,C/1/S,TRAPPIST-1e,3,False,...,181,585,Meratz Caltilter,True,7418,middle,0021,01,False,male
4,0023_01,Earth,False,F,F5,S,F/5/S,TRAPPIST-1e,1,False,...,0,0,Brence Harperez,True,645,poor,0023,01,False,female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G,G1496,S,G/1496/S,TRAPPIST-1e,2,False,...,0,0,Jeron Peter,True,0,poor,9266,02,True,male
4273,9269_01,Earth,False,NaN,NaN,NaN,NaN,TRAPPIST-1e,3,False,...,10,144,Matty Scheron,True,1018,poor,9269,01,False,female
4274,9271_01,Mars,True,D,D296,P,D/296/P,55 Cancri e,2,False,...,0,0,Jayrin Pore,True,0,poor,9271,01,False,male
4275,9273_01,Europa,False,D,D297,P,D/297/P,TRAPPIST-1e,2,False,...,0,523,Kitakan Conale,True,3203,poor,9273,01,False,male


## 불린 타입 피처들 Object형으로 캐스팅

In [35]:
bool_data = [column for column in data.select_dtypes(["bool"])]

bool_data

['CryoSleep', 'VIP', 'Transported', 'Group']

In [36]:
data["VIP"] = data["VIP"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")
data["CryoSleep"] = data["CryoSleep"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")
data["Group"] = data["Group"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8678 entries, 0 to 8692
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PassengerId   8678 non-null   object
 1   HomePlanet    8678 non-null   object
 2   CryoSleep     8678 non-null   object
 3   Cabin1        8575 non-null   object
 4   Combi         8575 non-null   object
 5   Cabin3        8575 non-null   object
 6   Cabin         8575 non-null   object
 7   Destination   8678 non-null   object
 8   Age           8678 non-null   int64 
 9   VIP           8678 non-null   object
 10  RoomService   8678 non-null   int64 
 11  FoodCourt     8678 non-null   int64 
 12  ShoppingMall  8678 non-null   int64 
 13  Spa           8678 non-null   int64 
 14  VRDeck        8678 non-null   int64 
 15  Name          8478 non-null   object
 16  Transported   8678 non-null   bool  
 17  Total         8678 non-null   int64 
 18  RichPoor      8678 non-null   object
 19  GroupI

In [38]:
bool_test = [column for column in test.select_dtypes(["bool"])]

bool_test

['CryoSleep', 'VIP', 'Transported', 'Group']

In [39]:
test["VIP"] = test["VIP"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")
test["CryoSleep"] = test["CryoSleep"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")
test["Group"] = test["Group"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")

# Cabin 결측값들 제거

In [40]:
data.dropna(axis=0, inplace=True)

# 필요없는 features 제거

In [41]:
target = data['Transported']
data = data.drop(["PassengerId", "Name", "Cabin", "Combi", "Total", "GroupId", "GroupNo", "Transported",
                 'RoomService', 'FoodCourt','ShoppingMall', 'Spa', 'VRDeck'], axis = 1)
test = test.drop(["PassengerId", "Name", "Cabin", "Combi", "Total", "GroupId", "GroupNo", "Transported",
                 'RoomService', 'FoodCourt','ShoppingMall', 'Spa', 'VRDeck'], axis = 1)


# 데이터세트 확인 

In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8375 entries, 0 to 8692
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   HomePlanet   8375 non-null   object
 1   CryoSleep    8375 non-null   object
 2   Cabin1       8375 non-null   object
 3   Cabin3       8375 non-null   object
 4   Destination  8375 non-null   object
 5   Age          8375 non-null   int64 
 6   VIP          8375 non-null   object
 7   RichPoor     8375 non-null   object
 8   Group        8375 non-null   object
 9   Gender       8375 non-null   object
dtypes: int64(1), object(9)
memory usage: 719.7+ KB


In [43]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   HomePlanet   4277 non-null   object
 1   CryoSleep    4277 non-null   object
 2   Cabin1       4214 non-null   object
 3   Cabin3       4214 non-null   object
 4   Destination  4277 non-null   object
 5   Age          4277 non-null   int64 
 6   VIP          4277 non-null   object
 7   RichPoor     4277 non-null   object
 8   Group        4277 non-null   object
 9   Gender       4277 non-null   object
dtypes: int64(1), object(9)
memory usage: 334.3+ KB


# data, target, test 라벨 인코딩

In [44]:
from sklearn.preprocessing import LabelEncoder

In [45]:
le = LabelEncoder()
new_cat = pd.DataFrame()
for i in range(0, 10):
    le.fit(data.iloc[:, i])
    le_encoded = le.transform(data.iloc[:, i])
    new_cat[data.columns[i]] = le_encoded

In [46]:
new_cat.head()

,HomePlanet,CryoSleep,Cabin1,Cabin3,Destination,Age,VIP,RichPoor,Group,Gender
0,1,0,1,0,2,3,0,1,0,1
1,0,0,5,1,2,1,0,1,0,0
2,1,0,0,1,2,3,1,0,1,1
3,1,0,0,1,2,2,0,0,1,1
4,0,0,5,1,2,0,0,1,0,0


In [47]:
data = new_cat

In [48]:
le = LabelEncoder()
new_test = pd.DataFrame()
for i in range(0, 10):
    le.fit(test.iloc[:, i])
    le_encoded = le.transform(test.iloc[:, i])
    new_test[test.columns[i]] = le_encoded

In [49]:
new_test.head()

,HomePlanet,CryoSleep,Cabin1,Cabin3,Destination,Age,VIP,RichPoor,Group,Gender
0,0,1,6,1,2,2,0,1,0,0
1,0,0,5,1,2,1,0,1,0,0
2,1,1,2,1,0,2,0,1,0,0
3,1,0,2,1,2,3,0,0,0,1
4,0,0,5,1,2,1,0,1,0,0


In [50]:
test = new_test

In [51]:
new_target = le.fit_transform(target)

In [52]:
new_target

array([0, 1, 0, ..., 1, 0, 1], dtype=int64)

In [53]:
target = new_target

# 데이터 세트 분리 전 데이터 담은 변수명과 타겟을 담은 변수명 확인 
- data = data 
- target = lable

# 데이터셋 분리

In [54]:
X_train, X_test, y_train, y_test = train_test_split(data, target, shuffle = True, random_state = 109)

# CatBoostClassifier (Category+Boosting)

- 결정트리의 gradient boosting을 위한 알고리즘
- 딥러닝 프레임워크(텐서플로우와 CoreML)과 통합됨
- 범주형 변수를 사용할 수 있음(예, 음성,문자,이미지 등)
- 범주형 변수 전처리와 오버피팅 문제 해결에 초점을 둠

- 장점
1. 다른 머신러닝 모델들과 달리 범주형 변수를 전처리 과정을 거치지 않고 사용할 수 있음 
<br> -원-핫 인코딩을 하거나 selectKbest를 통해 변수를 추출하지 않아도 원데이터를 그대로 다룰 수 있음 

2. Catboost 알고리즘은 변수들간의 상관관계가 높은 피처들을 파악해서 사용할 변수를 하나만 선택하는 능력을 가지고 있음 
3. Catboost 알고리즘은 파라미터 조정이 필수적인 다른 부스팅 기법들과 달리, 초기 파라미터 설정값만으로도 효율적인 예측이 가능 


https://cypision.github.io/machine-learning/Titanic_Catboost/#2catboosting-modeling---cv-%ED%8F%89%EA%B0%80
<br>https://naver.me/xI28QodH

## CatBoost 설치

In [55]:
!pip install catboost

## 필요한 라이브러리 불러오기

In [56]:
from catboost import CatBoostClassifier,cv

##  데이터세트 확인

In [57]:
print(data.shape,target.shape)
print(test.shape)

(8375, 10) (8375,)
(4277, 10)


In [58]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8375 entries, 0 to 8374
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   HomePlanet   8375 non-null   int32
 1   CryoSleep    8375 non-null   int32
 2   Cabin1       8375 non-null   int32
 3   Cabin3       8375 non-null   int32
 4   Destination  8375 non-null   int32
 5   Age          8375 non-null   int64
 6   VIP          8375 non-null   int32
 7   RichPoor     8375 non-null   int32
 8   Group        8375 non-null   int32
 9   Gender       8375 non-null   int32
dtypes: int32(9), int64(1)
memory usage: 360.0 KB


In [59]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   HomePlanet   4277 non-null   int32
 1   CryoSleep    4277 non-null   int32
 2   Cabin1       4277 non-null   int32
 3   Cabin3       4277 non-null   int32
 4   Destination  4277 non-null   int32
 5   Age          4277 non-null   int64
 6   VIP          4277 non-null   int32
 7   RichPoor     4277 non-null   int32
 8   Group        4277 non-null   int32
 9   Gender       4277 non-null   int32
dtypes: int32(9), int64(1)
memory usage: 183.9 KB


In [60]:
cat_features = list(range(0, data.shape[1])) # 컬럼의 인덱스 값 리스트화 
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [61]:
clf = CatBoostClassifier(
    iterations=50,
    random_seed=42,
    learning_rate=0.5,
    custom_loss=['AUC', 'Accuracy']
)

clf.fit(
    X_train, y_train,
    eval_set=(X_test, y_test), 
    cat_features=cat_features, 
    verbose=False,
)

print('CatBoost model is fitted: ' + str(clf.is_fitted()))
print('CatBoost model parameters:')
print(clf.get_params())

CatBoost model is fitted: True
CatBoost model parameters:
{'iterations': 50, 'learning_rate': 0.5, 'random_seed': 42, 'custom_loss': ['AUC', 'Accuracy']}


## get_clf_eval 텍스트북에 나온 평가함수 정의

In [62]:
def get_clf_eval(y_test, pred=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도 : {0:.3f}, 정밀도 : {1:.3f}, 재현율 : {2:.3f}, F1 : {3:.3f}'.format(accuracy, precision, recall, f1))

In [63]:
cat_pred = clf.predict(X_train)

In [64]:
get_clf_eval(y_train, cat_pred)

오차 행렬
[[2620  491]
 [1126 2044]]
정확도 : 0.743, 정밀도 : 0.806, 재현율 : 0.645, F1 : 0.717


In [65]:
cat_pred = clf.predict(X_test)
get_clf_eval(y_test,cat_pred)

오차 행렬
[[871 164]
 [370 689]]
정확도 : 0.745, 정밀도 : 0.808, 재현율 : 0.651, F1 : 0.721


## 마지막 최종 테스터 데이터세트 검증하기

In [66]:
sample = pd.read_csv('sample_submission.csv',sep=',')

In [67]:
prediction = clf.predict(test)
submission = pd.DataFrame({
        "PassengerId": sample["PassengerId"],
        "Transported": prediction})

In [68]:
submission['Transported'].replace({1:True,0:False},inplace=True)

In [69]:
submission.to_csv('submission_sparta.csv',index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\k_digital\\source\\sparta_data\\submission_sparta.csv'